# In the main branch, this file should be no executed and with no parameters set.
# In the development branch, it must be fully executed and must include tests.

GOAL: generate an SQLite DB from the CSV files at a given path.

In [1]:
import numpy as np
import pandas as pd
import os
import sqlite3

## Parameters

In [2]:
# Set the path and the file name of the SQLite DB to store the results.
#
# Example:
# 
# path = '/home/laercio/Dropbox/pesquisa/2017-voterModel/'
# path += 'code/resultados_e_analises/'
# path += '002__bubble_filtering__8_neighbors_squared_network/'
# path += 'raw_results/'     # Too long path... does not fit my screen.
#
# db_filename = 'test.db'

path = '/home/laercio/Dropbox/pesquisa/2017-voterModel/'
path += 'code/resultados_e_analises/'
path += '002__bubble_filtering__8_neighbors_squared_network/'
path += 'raw_results/'     # Too long path... does not fit my screen.

db_filename = 'test.db'

## Main

In [3]:
# fl - The file list of csv files at the path.
fl = os.listdir(path)
fl = [i for i in fl if i.endswith('.csv')]
fl.sort()

In [4]:
%%time

# concatenate all the csv files.

full = pd.DataFrame(columns=['v', 'N', 'q', 'net', 'rep', 
                             'mT', 'm2T', 'm4T', 'mT_noMod', 
                             'std_', 'std_noMod'])

for f in fl:
    df = pd.read_csv(path + f, index_col=0)
    v = float(f[2:6])
    df_temp = pd.DataFrame(data=v*np.ones(len(df)), index=df.index, 
                           columns=['v'])
    df = pd.concat([df_temp, df], axis=1)
    full = pd.concat([full, df])

CPU times: user 208 ms, sys: 16.4 ms, total: 224 ms
Wall time: 503 ms


In [5]:
# Have a glance at the concatenation result.
full.head()

,v,N,q,net,rep,mT,m2T,m4T,mT_noMod,std_,std_noMod
0,0.25,400,0.0000,-1,1,0.206236,0.063894,0.010397,-0.001027,0.146153,0.252770
1,0.25,400,0.0005,-1,1,0.201640,0.061150,0.009585,-0.004404,0.143147,0.247245
2,0.25,400,0.0010,-1,1,0.204047,0.062297,0.009877,0.008375,0.143741,0.249452
3,0.25,400,0.0015,-1,1,0.201144,0.060938,0.009605,-0.001429,0.143104,0.246852
4,0.25,400,0.0020,-1,1,0.199880,0.059965,0.009207,-0.005571,0.141466,0.244814


In [6]:
full.shape

# Expected: (9483, 11)

(9483, 11)

In [7]:
# Delete results with standard deviation bigger than one 
# (they are spurious).

full = full[full.std_ <= 1]

In [8]:
full.shape

# Expected: (9440, 11)

(9440, 11)

In [9]:
# Write the pandas data frame to the SQL db.

conn = sqlite3.connect(db_filename)
full.to_sql('temporal_mean_results', conn)
conn.close()

## Testing

In [10]:
# Read the sqlite db to a pandas data frame.

conn = sqlite3.connect(db_filename)

test = pd.read_sql('SELECT * FROM temporal_mean_results', conn,
                   index_col = 'index'
                  )
conn.close()

In [11]:
test.shape == (9440, 11)

True

In [12]:
test.head()

,v,N,q,net,rep,mT,m2T,m4T,mT_noMod,std_,std_noMod
index,,,,,,,,,,,
0,0.25,400,0.0000,-1,1,0.206236,0.063894,0.010397,-0.001027,0.146153,0.252770
1,0.25,400,0.0005,-1,1,0.201640,0.061150,0.009585,-0.004404,0.143147,0.247245
2,0.25,400,0.0010,-1,1,0.204047,0.062297,0.009877,0.008375,0.143741,0.249452
3,0.25,400,0.0015,-1,1,0.201144,0.060938,0.009605,-0.001429,0.143104,0.246852
4,0.25,400,0.0020,-1,1,0.199880,0.059965,0.009207,-0.005571,0.141466,0.244814


In [13]:
# Check that the means of columns are OK.
# The difference must be zero.

for i in full.columns:
    print('%9s' % i, np.mean(full[i] - test[i]))

        v 0.0
        N 0.0
        q 0.0
      net 0.0
      rep 0.0
       mT 0.0
      m2T 0.0
      m4T 0.0
 mT_noMod 0.0
     std_ 0.0
std_noMod 0.0


In [14]:
# Check if the data frames have the same columns.

(full.columns == test.columns).mean()

1.0

In [15]:
# Check if the data frames have the same index.

(full.index == test.index).mean()

1.0